In [2]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

C:\Users\Javi\AppData\Local\Temp\ipykernel_73012\3777615979.py:1: DeprecationWarning: Importing display from IPython.core.display is deprecated since IPython 7.14, please import from IPython display
  from IPython.core.display import display, HTML


# Lab | Natural Language Processing
### SMS: SPAM or HAM

### Let's prepare the environment

In [22]:
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression

- Read Data for the Fraudulent Email Kaggle Challenge
- Reduce the training set to speed up development. 

In [4]:
## Read Data for the Fraudulent Email Kaggle Challenge
email_df = pd.read_csv("E:\IronHack\Week 4\Lab_3 - Natural Language Processing\lab-natural-language-processing\data\kg_train.csv",encoding='latin-1')

# Reduce the training set to speed up development. 
#TODO Modify for final system
email_df = email_df.head(1000)
print(email_df.shape)
email_df.fillna("",inplace=True)
email_df.head(10)

(1000, 2)


<>:2: SyntaxWarning: invalid escape sequence '\I'
<>:2: SyntaxWarning: invalid escape sequence '\I'
C:\Users\Javi\AppData\Local\Temp\ipykernel_73012\2609927970.py:2: SyntaxWarning: invalid escape sequence '\I'
  email_df = pd.read_csv("E:\IronHack\Week 4\Lab_3 - Natural Language Processing\lab-natural-language-processing\data\kg_train.csv",encoding='latin-1')


,text,label
0,"DEAR SIR, STRICTLY A PRIVATE BUSINESS PROPOSAL...",1
1,Will do.,0
2,Nora--Cheryl has emailed dozens of memos about...,0
3,Dear Sir=2FMadam=2C I know that this proposal ...,1
4,fyi,0
5,sure -- bottom line - you need a special secur...,0
6,"Dear Sir,I am Engr. Ugo Nzego with the Enginee...",1
7,Abedin Huma <AbedinH@state.gov>Saturday Novemb...,0
8,There is an Oct 16th George Marshall event at ...,0
9,<P>1 25% for you as the account owner <BR>2 65...,1


### Let's divide the training and test set into two partitions

In [5]:
# Defining Features and Target
X = email_df["text"]
y = email_df["label"]

# Splitting the data into train and test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=12)

## Data Preprocessing

In [6]:
import string
from nltk.corpus import stopwords
print(string.punctuation)
print(stopwords.words("english")[100:110])
from nltk.stem.snowball import SnowballStemmer
snowball = SnowballStemmer('english')

!"#$%&'()*+,-./:;<=>?@[\]^_`{|}~
['here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each']


## Now, we have to clean the html code removing words

- First we remove inline JavaScript/CSS
- Then we remove html comments. This has to be done before removing regular tags since comments can contain '>' characters
- Next we can remove the remaining tags

In [7]:
from bs4 import BeautifulSoup, Comment

def clean_css(email_text):
    soup = BeautifulSoup(email_text, 'html.parser')
    for style in soup.findAll(attrs={'style': True}):
        style.replace_with('')
    for script in soup(["script", "style"]):
        script.extract()
    for element in soup(text=lambda text: isinstance(text, Comment)):
        element.extract()
    for tag in soup.find_all():
        tag.unwrap()
        
    return str(soup)

X_train_no_css = X_train.apply(clean_css)
X_test_no_css = X_test.apply(clean_css)
X_train_no_css.head(10)

C:\Users\Javi\AppData\Local\Temp\ipykernel_73012\92197393.py:4: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  soup = BeautifulSoup(email_text, 'html.parser')
C:\Users\Javi\AppData\Local\Temp\ipykernel_73012\92197393.py:4: MarkupResemblesLocatorWarning: The input looks more like a URL than markup. You may want to use an HTTP client like requests to get the document behind the URL, and feed that document to Beautiful Soup.
  soup = BeautifulSoup(email_text, 'html.parser')


56     Tom harkin called office twice duringCell:Home:AM
955    Dear sir=2C I got your contact in cause of a s...
231                                                   18
738                               She is on her cell.###
740                         Pls keep the updates coming!
566    Pis print.H Wednesday. October 17 2012 5.48 PM...
681                                                  Fyi
463                                                 Yes!
866    Sorry - been on calls all day re coordination ...
11     Dear Friend,My name is Edward Moore QC.Princip...
Name: text, dtype: object

- Remove all the special characters
    
- Remove numbers
    
- Remove all single characters
 
- Remove single characters from the start

- Substitute multiple spaces with single space

- Remove prefixed 'b'

- Convert to Lowercase

In [8]:
def clean_text(text):
        
    text = ''.join(f' {e} ' if e in string.punctuation else e for e in text)  # Add spaces around special characters
    text = ''.join(e for e in text if e.isalpha() or e.isspace())  # Remove special characters and numbers
    text = ' '.join(word for word in text.split() if len(word) > 1)  # Remove single characters
    text = text.lstrip()  # Remove single characters from the start
    text = ' '.join(text.split())  # Substitute multiple spaces with single space
    text = text.replace('b ', '')  # Remove prefixed 'b'
    text = text.lower()  # Convert to lowercase
    text = text.strip()  # Remove leading/trailing whitespace
    
    return text

X_train_clean = X_train_no_css.apply(clean_text)
X_test_clean = X_test_no_css.apply(clean_text)
X_train_clean.head(10)

56     tom harkin called office twice duringcell home am
955    dear sir got your contact in cause of seriouse...
231                                                     
738                                   she is on her cell
740                          pls keep the updates coming
566                    pis print wednesday october pmbfw
681                                                  fyi
463                                                  yes
866    sorry been on calls all day re coordination fo...
11     dear friend my name is edward moore qc princip...
Name: text, dtype: object

## Now let's work on removing stopwords
Remove the stopwords.

In [24]:
stop_words = set(stopwords.words('english'))  # Get the English stopwords

# Remove stopwords
X_train_no_sw = X_train_clean.apply(lambda x: ' '.join(word for word in x.split() if word not in stop_words))
X_test_no_sw = X_test_clean.apply(lambda x: ' '.join(word for word in x.split() if word not in stop_words))
X_train_no_sw.head(10)

# Removing rows that have became empty after pre-processing
#X_train_no_sw = X_train_no_sw[X_train_no_sw != '']
#X_test_no_sw = X_test_no_sw[X_test_no_sw != '']

56        tom harkin called office twice duringcell home
955    dear sir got contact cause seriouse search rel...
231                                                     
738                                                 cell
740                              pls keep updates coming
566                    pis print wednesday october pmbfw
681                                                  fyi
463                                                  yes
866            sorry calls day coordination haitihow sun
11     dear friend name edward moore qc principal par...
Name: text, dtype: object

## Tame Your Text with Lemmatization
Break sentences into words, then use lemmatization to reduce them to their base form (e.g., "running" becomes "run"). See how this creates cleaner data for analysis!

In [25]:
# Your code
import nltk
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.corpus import wordnet

# Initialize the WordNetLemmatizer
lemmatizer = WordNetLemmatizer()

def get_wordnet_pos(word):
    """Map POS tag to first character lemmatize() accepts"""
    tag = nltk.pos_tag([word])[0][1][0].upper()
    tag_dict = {"J": wordnet.ADJ,
                "N": wordnet.NOUN,
                "V": wordnet.VERB,
                "R": wordnet.ADV}
    
    return tag_dict.get(tag, wordnet.NOUN) # returns the word type (Noun if we have not found)

def lemmatize_email(text):

    # tokenize the text by splitting on spaces
    tokens = text.split()

    # lemmatize tokens
    lem_tokens = [lemmatizer.lemmatize(word, get_wordnet_pos(word)) for word in tokens]

    return ' '.join(lem_tokens)

X_train_lem = X_train_no_sw.apply(lemmatize_email)
X_test_lem = X_test_no_sw.apply(lemmatize_email)

## Bag Of Words
Let's get the 10 top words in ham and spam messages (**EXPLORATORY DATA ANALYSIS**)

In [26]:
# Vectorizing X_train using Bag of Words with CountVectorizer
bow_vectorizer = CountVectorizer()
X_train_bow = bow_vectorizer.fit_transform(X_train_lem)
#X_test_bow = bow_vectorizer.transform(X_test_lem)

In [27]:
# Counting word frequencies
word_freqs = X_train_bow.toarray().sum(axis=0)

# Sort words by frequency in descending order
sorted_words = sorted(zip(
    bow_vectorizer.get_feature_names_out(), word_freqs), key=lambda x: x[1], reverse=True)


# Print the top 10 words
for word, freq in sorted_words[:10]:
    print(f"{word}: {freq}")

money: 795
account: 697
bank: 634
fund: 593
mr: 478
country: 451
transaction: 436
business: 433
transfer: 428
million: 362


## Extra features

In [28]:
data_train = pd.concat([X_train_lem, y_train], axis=1)
data_val = pd.concat([X_test_lem, y_test], axis=1)

In [ ]:
# We add to the original dataframe two additional indicators (money symbols and suspicious words).
money_simbol_list = "|".join(["euro","dollar","pound","€","$"])
suspicious_words = "|".join(["free","cheap","sex","money","account","bank","fund","transfer","transaction","win","deposit","password"])

data_train['money_mark'] = data_train['preprocessed_text'].str.contains(money_simbol_list)*1
data_train['suspicious_words'] = data_train['preprocessed_text'].str.contains(suspicious_words)*1
data_train['text_len'] = data_train['preprocessed_text'].apply(lambda x: len(x)) 

data_val['money_mark'] = data_val['preprocessed_text'].str.contains(money_simbol_list)*1
data_val['suspicious_words'] = data_val['preprocessed_text'].str.contains(suspicious_words)*1
data_val['text_len'] = data_val['preprocessed_text'].apply(lambda x: len(x)) 

data_train.head()

## How would work the Bag of Words with Count Vectorizer concept?

In [ ]:
"""
With CountVectorizer a matrix is created where each row represents a sentence,\
    and each column represents a unique word (or token) in the corpus.
Then CountVectorizer counts the frequency of each word in each sentence\
    and stores it in the matrix.
The resulting matrix represents the sentences as numerical vectors, where each element\
    in the vector corresponds to the frequency of a particular word in the document.
    This allows for the comparison and analysis of text using various machine learning algorithms.
"""

## TD-IDF

- Load the vectorizer

- Vectorize all dataset

- print the shape of the vetorized dataset

In [30]:
# Vectorizing the dataset using TF-IDF
tfidf_vectorizer = TfidfVectorizer(ngram_range=(1,2))
X_train_tfidf = tfidf_vectorizer.fit_transform(X_train_lem)
X_test_tfidf = tfidf_vectorizer.transform(X_test_lem)

print(X_train_tfidf.shape)

(800, 76282)


## And the Train a Classifier?

In [31]:
# Logistic Regression
model = LogisticRegression(max_iter=1000).fit(X_train_tfidf, y_train)
y_pred = model.predict(X_test_tfidf)

### Extra Task - Implement a SPAM/HAM classifier

https://www.kaggle.com/t/b384e34013d54d238490103bc3c360ce

The classifier can not be changed!!! It must be the MultinimialNB with default parameters!

Your task is to find the **best feature representation**.

You can work with teams of two persons (recommended).

In [ ]:
# Your code